# Projeto 2 - Ciência dos Dados

## Integrantes:
* Gabriela Kimi
* Luiza Ehrenberger
* Pedro Barão
* Rafael Paolino

## Objetivo: usar métodos de regressão para prever o preço de carros com base em suas características

In [14]:
#Bibliotecas

%matplotlib notebook

import pandas as pd
import numpy as np
from scipy.stats import norm, probplot
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols

from sklearn.tree import DecisionTreeRegressor 

from sklearn.preprocessing import OneHotEncoder

from mpl_toolkits.mplot3d import Axes3D

from IPython.display import display

import datetime

import seaborn as sns

### Função de regressão linear:

In [15]:
def regress(Y,X):
    '''
    Y: coluna do DataFrame utilizada como variável resposta (TARGET)
    X: coluna(s) do DataFrame utilizadas como variável(is) explicativas (FEATURES)
    '''
    X_cp = sm.add_constant(X)
    model = sm.OLS(Y,X_cp)
    results = model.fit()
    
    return results

### DataFrame de base de dados

In [16]:
data = pd.read_csv("data.csv")

In [17]:
data.head(3)

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350


In [18]:
data.columns

Index(['Make', 'Model', 'Year', 'Engine Fuel Type', 'Engine HP',
       'Engine Cylinders', 'Transmission Type', 'Driven_Wheels',
       'Number of Doors', 'Market Category', 'Vehicle Size', 'Vehicle Style',
       'highway MPG', 'city mpg', 'Popularity', 'MSRP'],
      dtype='object')

### Remoção de linhas com valores faltando:

In [19]:
data=data.dropna()
data.isnull().sum()

Make                 0
Model                0
Year                 0
Engine Fuel Type     0
Engine HP            0
Engine Cylinders     0
Transmission Type    0
Driven_Wheels        0
Number of Doors      0
Market Category      0
Vehicle Size         0
Vehicle Style        0
highway MPG          0
city mpg             0
Popularity           0
MSRP                 0
dtype: int64

In [20]:
data.iloc[:,0].size

8084

In [21]:
train = data.sample(7200)

In [22]:
train=train[[ 'Year', 'Engine Fuel Type', 'Engine HP',
       'Engine Cylinders', 'Transmission Type', 'Driven_Wheels',
       'Number of Doors', 'Vehicle Size',
       'highway MPG', 'city mpg', 'Popularity', 'MSRP']]

### Codificação de colunas categóricas para integer e hot encode:


In [23]:
train_2=pd.get_dummies(train)
train_2

,Year,Engine HP,Engine Cylinders,Number of Doors,highway MPG,city mpg,Popularity,MSRP,Engine Fuel Type_diesel,Engine Fuel Type_electric,...,Transmission Type_DIRECT_DRIVE,Transmission Type_MANUAL,Transmission Type_UNKNOWN,Driven_Wheels_all wheel drive,Driven_Wheels_four wheel drive,Driven_Wheels_front wheel drive,Driven_Wheels_rear wheel drive,Vehicle Size_Compact,Vehicle Size_Large,Vehicle Size_Midsize
1379,2014,220.0,4.0,4.0,27,20,3105,43300,0,0,...,0,0,0,1,0,0,0,0,0,1
10541,2016,280.0,6.0,4.0,23,17,873,46745,0,0,...,0,0,0,1,0,0,0,0,0,1
4187,2010,177.0,4.0,4.0,31,34,5657,29860,0,0,...,0,0,0,0,0,1,0,1,0,0
3745,2013,255.0,8.0,3.0,15,11,5657,36545,0,0,...,0,0,0,0,0,0,1,0,0,1
5383,2015,210.0,4.0,4.0,34,25,873,28385,0,0,...,0,1,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10498,2016,290.0,6.0,4.0,34,21,204,42600,0,0,...,0,0,0,0,0,1,0,0,0,1
744,1991,162.0,4.0,4.0,20,17,870,2000,0,0,...,0,0,0,0,0,0,1,0,0,1
4397,2015,290.0,6.0,4.0,23,17,5657,35000,0,0,...,0,0,0,1,0,0,0,0,0,1
5087,2011,330.0,6.0,2.0,27,19,190,37150,0,0,...,0,0,0,0,0,0,1,0,0,1


In [24]:
train_2.columns

Index(['Year', 'Engine HP', 'Engine Cylinders', 'Number of Doors',
       'highway MPG', 'city mpg', 'Popularity', 'MSRP',
       'Engine Fuel Type_diesel', 'Engine Fuel Type_electric',
       'Engine Fuel Type_flex-fuel (premium unleaded recommended/E85)',
       'Engine Fuel Type_flex-fuel (premium unleaded required/E85)',
       'Engine Fuel Type_flex-fuel (unleaded/E85)',
       'Engine Fuel Type_premium unleaded (recommended)',
       'Engine Fuel Type_premium unleaded (required)',
       'Engine Fuel Type_regular unleaded',
       'Transmission Type_AUTOMATED_MANUAL', 'Transmission Type_AUTOMATIC',
       'Transmission Type_DIRECT_DRIVE', 'Transmission Type_MANUAL',
       'Transmission Type_UNKNOWN', 'Driven_Wheels_all wheel drive',
       'Driven_Wheels_four wheel drive', 'Driven_Wheels_front wheel drive',
       'Driven_Wheels_rear wheel drive', 'Vehicle Size_Compact',
       'Vehicle Size_Large', 'Vehicle Size_Midsize'],
      dtype='object')

In [ ]:
train_2.describe()

,Year,Engine HP,Engine Cylinders,Number of Doors,highway MPG,city mpg,Popularity,MSRP,Engine Fuel Type_diesel,Engine Fuel Type_electric,...,Transmission Type_DIRECT_DRIVE,Transmission Type_MANUAL,Transmission Type_UNKNOWN,Driven_Wheels_all wheel drive,Driven_Wheels_four wheel drive,Driven_Wheels_front wheel drive,Driven_Wheels_rear wheel drive,Vehicle Size_Compact,Vehicle Size_Large,Vehicle Size_Midsize
count,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7.200000e+03,7200.000000,7200.000000,...,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000
mean,2012.083472,273.946667,5.761111,3.416389,26.713889,19.605417,1507.425556,4.942052e+04,0.019306,0.001528,...,0.001667,0.210972,0.000417,0.266389,0.080278,0.360556,0.292778,0.376806,0.229861,0.393333
std,6.332839,114.745080,1.878483,0.897254,7.788304,7.079765,1415.474647,6.729822e+04,0.137606,0.039060,...,0.040794,0.408027,0.020410,0.442101,0.271742,0.480195,0.455069,0.484619,0.420773,0.488524
min,1990.000000,55.000000,0.000000,2.000000,12.000000,8.000000,2.000000,2.000000e+03,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2010.000000,190.000000,4.000000,2.000000,23.000000,16.000000,549.000000,2.587000e+04,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2015.000000,260.000000,6.000000,4.000000,26.000000,18.000000,1013.000000,3.499500e+04,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2016.000000,320.000000,6.000000,4.000000,30.000000,22.000000,2009.000000,4.874625e+04,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000
max,2017.000000,1001.000000,16.000000,4.000000,354.000000,137.000000,5657.000000,2.065902e+06,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Plotagem de gráficos scatter

In [13]:
sns.pairplot(train_2)

<IPython.core.display.Javascript object>

KeyboardInterrupt: 

### Regressão linear MMQ:

In [25]:
X= train_2[['Year', 'Engine HP', 'Engine Cylinders', 'Number of Doors',
       'highway MPG', 'city mpg', 'Popularity','Engine Fuel Type_diesel', 'Engine Fuel Type_electric',
       'Engine Fuel Type_flex-fuel (premium unleaded recommended/E85)',
       'Engine Fuel Type_flex-fuel (premium unleaded required/E85)',
       'Engine Fuel Type_flex-fuel (unleaded/E85)',
       'Engine Fuel Type_premium unleaded (recommended)',
       'Engine Fuel Type_premium unleaded (required)',
       'Engine Fuel Type_regular unleaded',
       'Transmission Type_AUTOMATED_MANUAL', 'Transmission Type_AUTOMATIC',
       'Transmission Type_DIRECT_DRIVE', 'Transmission Type_MANUAL',
       'Transmission Type_UNKNOWN', 'Driven_Wheels_all wheel drive',
       'Driven_Wheels_four wheel drive', 'Driven_Wheels_front wheel drive',
       'Driven_Wheels_rear wheel drive', 'Vehicle Size_Compact',
       'Vehicle Size_Large', 'Vehicle Size_Midsize']]
Y=train[['MSRP']]

results=regress(Y,X)
results.summary()

D:\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   MSRP   R-squared:                       0.511
Model:                            OLS   Adj. R-squared:                  0.509
Method:                 Least Squares   F-statistic:                     325.4
Date:                Thu, 25 Nov 2021   Prob (F-statistic):               0.00
Time:                        14:59:31   Log-Likelihood:                -88091.
No. Observations:                7200   AIC:                         1.762e+05
Df Residuals:                    7176   BIC:                         1.764e+05
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================
                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------
const                                                         -2.793e+05   1.35e+05     -2.065      0.039   -5.45e+05   -1.41e+04
Year                                                            193.1771    129.791      1.488      0.137     -61.251     447.605
Engine HP                                                       279.2068     12.571     22.210      0.000     254.563     303.850
Engine Cylinders                                               1.404e+04    695.078     20.202      0.000    1.27e+04    1.54e+04
Number of Doors                                               -1010.9296    846.933     -1.194      0.233   -2671.167     649.308
highway MPG                                                     -16.9776    138.256     -0.123      0.902    -288.000     254.045
city mpg                                                       1377.9035    183.947      7.491      0.000    1017.312    1738.494
Popularity                                                       -3.2038      0.439     -7.296      0.000      -4.065      -2.343
Engine Fuel Type_diesel                                       -1.839e+04   1.87e+04     -0.981      0.327   -5.51e+04    1.84e+04
Engine Fuel Type_electric                                      -7.69e+04   3.68e+04     -2.087      0.037   -1.49e+05   -4678.579
Engine Fuel Type_flex-fuel (premium unleaded recommended/E85) -3.323e+04   2.06e+04     -1.609      0.108   -7.37e+04    7243.776
Engine Fuel Type_flex-fuel (premium unleaded required/E85)    -2.116e+04   1.91e+04     -1.107      0.269   -5.87e+04    1.63e+04
Engine Fuel Type_flex-fuel (unleaded/E85)                     -4.035e+04   1.88e+04     -2.146      0.032   -7.72e+04   -3486.049
Engine Fuel Type_premium unleaded (recommended)               -3.688e+04   1.85e+04     -1.990      0.047   -7.32e+04    -549.188
Engine Fuel Type_premium unleaded (required)                  -1.851e+04   1.83e+04     -1.011      0.312   -5.44e+04    1.74e+04
Engine Fuel Type_regular unleaded                             -3.391e+04   1.82e+04     -1.862      0.063   -6.96e+04    1788.568
Transmission Type_AUTOMATED_MANUAL                             -2.32e+04   2.86e+04     -0.811      0.417   -7.93e+04    3.29e+04
Transmission Type_AUTOMATIC                                   -4.538e+04   2.88e+04     -1.576      0.115   -1.02e+05    1.11e+04
Transmission Type_DIRECT_DRIVE                                -5.397e+04   3.96e+04     -1.363      0.173   -1.32e+05    2.37e+04
Transmission Type_MANUAL                                       -5.26e+04   2.84e+04     -1.855      0.064   -1.08e+05    2982.820
Transmission Type_UNKNOWN                                     -1.042e+05   3.64e+04     -2.86

### Remoção de colunas com valor p > 10%:

In [ ]:
X= train_2[['Engine HP', 'Engine Cylinders','city mpg', 'Popularity',
       'Engine Fuel Type_flex-fuel (unleaded/E85)',
       'Engine Fuel Type_premium unleaded (recommended)',
       'Engine Fuel Type_regular unleaded',
       'Transmission Type_AUTOMATED_MANUAL',
       'Transmission Type_DIRECT_DRIVE', 'Transmission Type_MANUAL',
       'Transmission Type_UNKNOWN', 'Driven_Wheels_all wheel drive',
       'Driven_Wheels_four wheel drive', 'Driven_Wheels_front wheel drive',
       'Driven_Wheels_rear wheel drive', 'Vehicle Size_Compact',
       'Vehicle Size_Large', 'Vehicle Size_Midsize']]

Y=train_2[['MSRP']]

results=regress(Y,X)
results.summary()

D:\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   MSRP   R-squared:                       0.514
Model:                            OLS   Adj. R-squared:                  0.513
Method:                 Least Squares   F-statistic:                     474.6
Date:                Thu, 25 Nov 2021   Prob (F-statistic):               0.00
Time:                        14:53:54   Log-Likelihood:                -88141.
No. Observations:                7200   AIC:                         1.763e+05
Df Residuals:                    7183   BIC:                         1.764e+05
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
===================================================================================================================
                                                      coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------
const                                           -7.858e+04   3392.300    -23.165      0.000   -8.52e+04   -7.19e+04
Engine HP                                         291.4546     10.546     27.638      0.000     270.782     312.127
Engine Cylinders                                 1.374e+04    649.352     21.167      0.000    1.25e+04     1.5e+04
city mpg                                         1425.2559    140.479     10.146      0.000    1149.875    1700.636
Popularity                                         -3.1423      0.437     -7.194      0.000      -3.999      -2.286
Engine Fuel Type_flex-fuel (unleaded/E85)       -2.079e+04   2545.657     -8.167      0.000   -2.58e+04   -1.58e+04
Engine Fuel Type_premium unleaded (recommended) -1.841e+04   1936.137     -9.511      0.000   -2.22e+04   -1.46e+04
Engine Fuel Type_regular unleaded               -1.518e+04   1820.526     -8.340      0.000   -1.88e+04   -1.16e+04
Transmission Type_AUTOMATED_MANUAL               2.366e+04   2525.110      9.370      0.000    1.87e+04    2.86e+04
Transmission Type_DIRECT_DRIVE                  -6.725e+04   1.81e+04     -3.716      0.000   -1.03e+05   -3.18e+04
Transmission Type_MANUAL                        -7242.6689   1743.377     -4.154      0.000   -1.07e+04   -3825.137
Transmission Type_UNKNOWN                       -5.904e+04   2.91e+04     -2.032      0.042   -1.16e+05   -2070.789
Driven_Wheels_all wheel drive                    -1.18e+04   1336.296     -8.830      0.000   -1.44e+04   -9180.108
Driven_Wheels_four wheel drive                  -3.155e+04   2044.978    -15.428      0.000   -3.56e+04   -2.75e+04
Driven_Wheels_front wheel drive                 -9125.0229   1542.669     -5.915      0.000   -1.21e+04   -6100.938
Driven_Wheels_rear wheel drive                  -2.611e+04   1419.018    -18.399      0.000   -2.89e+04   -2.33e+04
Vehicle Size_Compact                             -1.18e+04   1502.240     -7.857      0.000   -1.47e+04   -8857.649
Vehicle Size_Large                              -3.422e+04   1769.425    -19.337      0.000   -3.77e+04   -3.07e+04
Vehicle Size_Midsize                            -3.256e+04   1338.925    -24.321      0.000   -3.52e+04   -2.99e+04
==============================================================================
Omnibus:                    13692.624   Durbin-Watson:                   2.014
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         40365811.106
Skew:                          14.431   Prob(JB):                         0.00
Kurtosis:                     368.677   Cond. No.                     3.26e+19
==============================================================================

Notes:
[1] Standard Errors assume that the covariance ma

## Regressão em árvore:

In [ ]:
Y=train[['MSRP']]

X= train_2[['Year', 'Engine HP', 'Engine Cylinders', 'Number of Doors',
       'highway MPG', 'city mpg', 'Popularity','Engine Fuel Type_diesel', 'Engine Fuel Type_electric',
       'Engine Fuel Type_flex-fuel (premium unleaded recommended/E85)',
       'Engine Fuel Type_flex-fuel (premium unleaded required/E85)',
       'Engine Fuel Type_flex-fuel (unleaded/E85)',
       'Engine Fuel Type_premium unleaded (recommended)',
       'Engine Fuel Type_premium unleaded (required)',
       'Engine Fuel Type_regular unleaded',
       'Transmission Type_AUTOMATED_MANUAL', 'Transmission Type_AUTOMATIC',
       'Transmission Type_DIRECT_DRIVE', 'Transmission Type_MANUAL',
       'Transmission Type_UNKNOWN', 'Driven_Wheels_all wheel drive',
       'Driven_Wheels_four wheel drive', 'Driven_Wheels_front wheel drive',
       'Driven_Wheels_rear wheel drive', 'Vehicle Size_Compact',
       'Vehicle Size_Large', 'Vehicle Size_Midsize']]

regressor = DecisionTreeRegressor(random_state = 0) 

regressor.fit(X, Y)

DecisionTreeRegressor(random_state=0)

In [ ]:
y_pred = regressor.predict([[250]])

print("Predicted price: % d\n"% y_pred) 

ValueError: X has 1 features, but DecisionTreeRegressor is expecting 27 features as input.